In [1]:
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D



Using TensorFlow backend.


## Things to try
- New NN architecture
- Preprocess input
    - Black and White
    - 


In [2]:
def get_commaai_model(img_shape):
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    row, col, ch = img_shape  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

In [32]:
def get_nvidia_model(img_shape):
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    row, col, ch = img_shape  # camera format

    model = Sequential()
    model.add(Convolution2D(24, 5, 5, input_shape=(row, col, ch), subsample=(4, 4), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(ELU())
    model.add(Dense(1164))
    model.add(Dropout(0.2))
    model.add(ELU())
    model.add(Dense(100))
    model.add(ELU())
    model.add(Dense(50))
    model.add(ELU())
    model.add(Dense(10))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

In [3]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle


In [21]:

def get_data(log_file):
    
    df = pd.read_csv(log_file, sep=',')
    root_dir = '/'.join(log_file.split('/')[:-1])
    
    ex_img = root_dir + '/' + df.ix[0][0]
    img_shape = cv2.imread(ex_img).shape
    features = np.zeros([len(df), img_shape[0], img_shape[1], 1])
    labels = np.zeros([len(df), 1])

    for i, row in df.iterrows():
        img_path = root_dir + '/' + row[0]
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features[i] = np.expand_dims(gray, axis=3)
        labels[i] = row[3]
    
    return features, labels

features, labels = get_data('data/driving_log.csv')
print('Done')

# data = {
#     'features': features,
#     'labels': labels
# }
# with open('train.p', 'wr') as f:
#     pickle.dump(data, f)

Done


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)

In [33]:
print(X_train.shape[1:])

# model = get_commaai_model(X_train.shape[1:])

nvidia_model = get_nvidia_model(X_train.shape[1:])
history = nvidia_model.fit(X_train, y_train)


(160, 320, 1)
Epoch 1/10
5625/5625 [==============================] - 21s - loss: 0.7146 - acc: 0.5017       

In [34]:
print(history.history)

{'acc': [0.50168888899485276, 0.54488888892067799, 0.54488888884650333, 0.54488888899485266, 0.54488888880411779, 0.54488888880411779, 0.54488888892067799, 0.54488888889948528, 0.54488888899485266, 0.54488888901604549], 'loss': [0.71456917588777014, 0.022002631884151035, 0.017775581595136061, 0.015161736931403477, 0.013947556665208605, 0.012999479679101044, 0.012559408381912444, 0.012269821901453867, 0.011417306772039996, 0.011293383049633768]}


In [35]:
results = nvidia_model.evaluate(X_val, y_val)
print(results)

2411/2411 [==============================] - 3s      

In [36]:
nvidia_model.save('model.h5')